In [18]:
# Import packages
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
import math
import pickle
import joblib

In [2]:
# Read data
df_raw = pd.read_csv('C:/Users/karen/PycharmProjects/ycng228-project/.data/_SP500_data_all.csv',index_col = 0)
df_raw.index = pd.to_datetime(df_raw.index)
df_raw.index.name = 'date'
df_raw = df_raw.sort_index(axis = 0)

In [3]:
df_raw.head(5)

,open,high,low,close,adjclose,volume,ticker
date,,,,,,,
1970-03-25,6.781250,6.937500,6.781250,6.875000,0.261931,68400,ED
1970-03-25,0.341564,0.368313,0.340535,0.349794,0.158033,2041200,MCD
1970-03-25,8.084030,8.434241,8.054846,8.200767,1.285789,382912,IP
1970-03-25,1.753906,1.796875,1.753906,1.789063,0.158143,2720000,XOM
1970-03-25,15.750478,16.108988,15.750478,15.917782,3.698164,1303316,IBM


In [4]:
df = df_raw.set_index([df_raw.index,'ticker'])

In [5]:
df.head(5)

open       high        low      close  adjclose  \
date       ticker                                                         
1970-03-25 ED       6.781250   6.937500   6.781250   6.875000  0.261931   
           MCD      0.341564   0.368313   0.340535   0.349794  0.158033   
           IP       8.084030   8.434241   8.054846   8.200767  1.285789   
           XOM      1.753906   1.796875   1.753906   1.789063  0.158143   
           IBM     15.750478  16.108988  15.750478  15.917782  3.698164   

                    volume  
date       ticker           
1970-03-25 ED        68400  
           MCD     2041200  
           IP       382912  
           XOM     2720000  
           IBM     1303316

In [6]:
display(df)

open        high         low       close    adjclose  \
date       ticker                                                               
1970-03-25 ED        6.781250    6.937500    6.781250    6.875000    0.261931   
           MCD       0.341564    0.368313    0.340535    0.349794    0.158033   
           IP        8.084030    8.434241    8.054846    8.200767    1.285789   
           XOM       1.753906    1.796875    1.753906    1.789063    0.158143   
           IBM      15.750478   16.108988   15.750478   15.917782    3.698164   
...                       ...         ...         ...         ...         ...   
2022-10-10 DVN      71.889999   73.459999   70.019997   70.349998   70.349998   
           DXCM    101.904999  102.739998   99.660004  100.690002  100.690002   
           STE     172.000000  170.509995  166.929993  169.009995  169.009995   
           STZ     222.479996  222.679993  219.320007  221.615005  221.615005   
           ZTS     148.100006  147.945007  144.479996  146.229996  146.229996   

                    volume  
date       ticker           
1970-03-25 ED        68400  
           MCD     2041200  
           IP       382912  
           XOM     2720000  
           IBM     1303316  
...                    ...  
2022-10-10 DVN     8049050  
           DXCM    2241114  
           STE      254069  
           STZ      812788  
           ZTS     1000974  

[4074303 rows x 6 columns]

In [8]:
def filter_data(_df,_tickers=None,_dates=None,_columns=None):
    # filters dataframe with MULTIINDEX [date,ticker] by ticker and for range of dates, selecting data columns
    # by default will select all columns
    if _columns is None:
        _columns = _df.columns
    df_tmp = pd.DataFrame(_df[_df.index.get_level_values('ticker').isin(_tickers) & \
                  _df.index.get_level_values('date').isin(_dates)][_columns])
    
    return df_tmp


def get_last_values(_df,_pivot_date):
    # gets last value BEFORE the day of output
    df_tmp = _df.tail(2).head(1).reset_index().iloc[:, 2:]
    
    return df_tmp
    
    
def get_lags(_df,_lag=1):
    # filters one column dataframe to keep n last records using tail method
    # by default returns LAST row of dataframe
    df_tmp = _df.tail(_lag+1)
    
    return df_tmp


def fill_na(_df):
    # Fills NaaN cells with backfill method
    df_tmp = _df.fillna(method = 'bfill')
    
    return df_tmp


def create_diff(_df,_dropna=True):
    # calculates and returns difference between dataframe rows. Does so for ALL columns,
    # by default, resulting NaaN columns (oldest line) are then dropped
    if _dropna is True:
        df_tmp = _df.diff().dropna(axis='index')
    else:
        df_tmp = _df.diff()
    
    return df_tmp


def to_row(_df,_column,_column_names):
    # selects single column from dataframe, transposes it to a row.
    # returns one line dataframe with provided column names for easier append
    df_tmp = pd.DataFrame(_df[_column].values.T,columns=_column_names)
    
    return df_tmp


def create_lag_features(_df,_ticker,_pivot_date,_column='close',_lags=30,_diff=True):
    # creates input dataset and set of lag features based on one column dataframe, date of output (pivot date), column and
    # number of lag features to add
    dates = _df.index.get_level_values('date')[(_df.index.get_level_values('date') <= _pivot_date)]
    
    column_names = ['day' + str(i) for i in range(1,_lags+1)]
    column_names.append('output')
    column_names.append('ticker')
    column_names.append('date')
    df_lags = pd.DataFrame(columns = column_names)
    #create_diff(fill_na(get_lags(filter_data(df,tickers,dates,'close'),31)),True)
    #df_tmp = to_row(get_lags(_df,_lags+1),column_names[:-2])
    
    df_tmp1 = fill_na(get_lags(filter_data(_df,_ticker,dates,_column),_lags+1))
    if _diff is True:
        df_tmp1 = create_diff(df_tmp1)
    df_tmp2 = to_row(df_tmp1,_column,column_names[:-2])
    df_lags = pd.concat([df_lags, df_tmp2], ignore_index=True)                                                
    df_lags['ticker'] = df_tmp1.index.get_level_values('ticker').to_list()[0]
    df_lags['date'] = df_tmp1.index.get_level_values('date').to_list()[-1]
    
    return df_lags

def add_features(_df_tgt,df_src):
    df_tmp = pd.concat([_df_tgt, df_src],axis=1)
    
    return df_tmp


def tokenize_output(_df,_column='output'):
    df_tmp = _df
    df_tmp.loc[df_tmp[_column] <= 0, _column] = -1 
    df_tmp.loc[df_tmp[_column] >  0, _column] =  1 

    return df_tmp


def moving_average(_df,_pivot_date,_lags=30):
    mavg = 0
    return mavg


def create_input(_df, _earliest_pivot='2019-04-01',_latest_pivot='2022-04-30',_debug=False):
    # Creates training input for logistic regression
    if _debug is True:
        tickers = ['AMZN','AAPL','TSLA','MSFT','GOOGL']
        pivots = ['2022-04-29','2022-04-28','2022-04-27','2022-04-26','2022-04-25']
        dates = _df.index.get_level_values('date')[(_df.index.get_level_values('date') <= _latest_pivot)]
    else:
        tickers = _df.index.get_level_values('ticker')
        dates = _df.index.get_level_values('date')[(_df.index.get_level_values('date') <= _latest_pivot)]
        pivots = _df.index.get_level_values('date')[(_df.index.get_level_values('date') > _earliest_pivot) & \
                                                   (_df.index.get_level_values('date') <= _latest_pivot)]
    # Initialize input dataframe
    df_input = pd.DataFrame()
    
        
    for i in tickers:
        print('Processing ticker: '+str(i))
        # create initial ticker DF once
        df_i = filter_data(_df,_tickers=[i],_dates=dates)
        
        for j in pivots:
            # create lag features, format, for each pivot date
            df_j = create_lag_features(_df=df_i,_ticker=[i],_pivot_date=j,_column=['close'],_lags=30)
            # add feature(s)
            #df_j = pd.concat([df_j,get_last_values(filter_data(_df=df_i,_tickers=[i],_dates=dates, \
            #                                                     _columns=['volume','low','high']),\
            #                                         _pivot_date=j)],axis=1)
            # Initialize input columns for the first time 
            if len(df_input.columns)==0:
                df_input = pd.DataFrame(columns=df_j.columns)
                
            df_input = pd.concat([df_input, df_j], ignore_index=True)
    
    # Tokenize output column
    df_input = tokenize_output(df_input)
    
    # reorder columns
    lst = df_input.columns.to_list()
    lst.remove('output')
    lst.remove('ticker')
    lst.remove('date')
    lst.append('output')
    lst.append('ticker')
    lst.append('date')
    df_input = df_input.reindex(columns=lst)
    
    print('Done...')
    
    return df_input

In [17]:
# Baseline : Diff of closing price only.
df_train_0=create_input(df,_earliest_pivot='2021-04-01',_latest_pivot='2022-04-30')

Processing ticker: ED


KeyboardInterrupt: 

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day24,day25,day26,day27,day28,day29,day30,output,ticker,date
0,4.134995,4.011505,0.240997,3.397507,-1.481003,0.241501,1.123993,4.216995,0.324509,-3.014008,...,-4.117493,-5.701996,-3.945999,1.723999,-6.682999,-1.223999,6.429489,-1.0,AMZN,2022-04-29
1,5.737503,4.134995,4.011505,0.240997,3.397507,-1.481003,0.241501,1.123993,4.216995,0.324509,...,5.330490,-4.117493,-5.701996,-3.945999,1.723999,-6.682999,-1.223999,1.0,AMZN,2022-04-28
2,5.513504,5.737503,4.134995,4.011505,0.240997,3.397507,-1.481003,0.241501,1.123993,4.216995,...,1.078506,5.330490,-4.117493,-5.701996,-3.945999,1.723999,-6.682999,-1.0,AMZN,2022-04-27
3,-3.671509,5.513504,5.737503,4.134995,4.011505,0.240997,3.397507,-1.481003,0.241501,1.123993,...,-3.834503,1.078506,5.330490,-4.117493,-5.701996,-3.945999,1.723999,-1.0,AMZN,2022-04-26
4,-1.292999,-3.671509,5.513504,5.737503,4.134995,4.011505,0.240997,3.397507,-1.481003,0.241501,...,4.753494,-3.834503,1.078506,5.330490,-4.117493,-5.701996,-3.945999,1.0,AMZN,2022-04-25
5,1.029999,3.360001,1.400009,3.440002,1.389999,3.860001,0.649994,0.880005,3.360001,-1.190002,...,-0.169998,-0.809998,-4.630005,1.090012,-6.080002,-0.229996,7.069992,-1.0,AAPL,2022-04-29
6,4.500000,1.029999,3.360001,1.400009,3.440002,1.389999,3.860001,0.649994,0.880005,3.360001,...,2.329987,-0.169998,-0.809998,-4.630005,1.090012,-6.080002,-0.229996,1.0,AAPL,2022-04-28
7,4.470001,4.500000,1.029999,3.360001,1.400009,3.440002,1.389999,3.860001,0.649994,0.880005,...,-0.219986,2.329987,-0.169998,-0.809998,-4.630005,1.090012,-6.080002,-1.0,AAPL,2022-04-27
8,-4.110001,4.470001,4.500000,1.029999,3.360001,1.400009,3.440002,1.389999,3.860001,0.649994,...,-5.110001,-0.219986,2.329987,-0.169998,-0.809998,-4.630005,1.090012,-1.0,AAPL,2022-04-26
9,-3.790009,-4.110001,4.470001,4.500000,1.029999,3.360001,1.400009,3.440002,1.389999,3.860001,...,2.739990,-5.110001,-0.219986,2.329987,-0.169998,-0.809998,-4.630005,1.0,AAPL,2022-04-25
